In [19]:
from __future__ import division

In [20]:
import json
import pandas as pd
import numpy as np

In [21]:
from imret.query import Annotation, KnowledgeBase
from imret.dataset import Dataset

In [22]:
with open('../data/prover/map_prover-0900_retrieved.json', 'r') as fp:
    retrieved = json.load(fp)

In [23]:
db = Dataset('../data/datasets/', 'test', '../data/images/')
# qa = Annotation('../data/query/test_anno/')
qa = Annotation('../data/corrected_test_anno/')

In [24]:
df = pd.read_csv('../data/query/query_equivalence.csv')
df.dropna(inplace=True)
queries = dict(zip(df['Original'], df['Equivalent']))

In [25]:
def apk(expected, predicted, k=4317):
    retrieved = np.array([ret in expected for ret in predicted[:k]]).astype(np.int32)
    if retrieved.sum() == 0:
        return 0.0

    scores = np.arange(len(retrieved)) + 1
    valids = np.where(retrieved > 0)
    return np.sum(np.cumsum(retrieved[valids]) / scores[valids].astype(np.float32)) / float(min(len(expected), k))

In [28]:
scores = []
for query in sorted(qa.db):
    try:
        alias = queries[query]
        score = apk(qa.db[query], retrieved[alias])
        if score > 0.01:
            scores.append(score)
            print "\\textit{{{}}} & \\textit{{{}}} & {:4d} & \t {:4d} & \t {:.2f} \\\\".format(query, alias, len(retrieved[alias]), 
                                                                  len(qa.db[query]), score)
    except KeyError:
        continue
        
print "mAP {:.2f}%".format(100 * np.mean(scores))

\textit{airplane-in_front_of-building} & \textit{building-behind-airplane} &   16 & 	    6 & 	 0.32 \\
\textit{armchair-in_front_of-wall} & \textit{wall-behind-armchair} &   74 & 	   44 & 	 0.07 \\
\textit{bench-in_front_of-wall} & \textit{wall-behind-bench} &   27 & 	   34 & 	 0.02 \\
\textit{books-inside_of-bookcase} & \textit{books-in-bookcase} &    4 & 	   21 & 	 0.05 \\
\textit{building-behind-wall} & \textit{wall-in_front_of-building} &  155 & 	   84 & 	 0.05 \\
\textit{curtain-above-floor} & \textit{floor-below-curtain} &  153 & 	  148 & 	 0.08 \\
\textit{fence-in_front_of-tree} & \textit{tree-behind-fence} &   96 & 	   37 & 	 0.02 \\
\textit{floor-under-sky} & \textit{sky-above-floor} &  496 & 	   46 & 	 0.03 \\
\textit{grass-below-window} & \textit{window-above-grass} &   25 & 	   99 & 	 0.06 \\
\textit{ground-under-sky} & \textit{sky-above-ground} &  187 & 	  216 & 	 0.09 \\
\textit{road-below-building} & \textit{building-above-road} &   54 & 	  302 & 	 0.01 \\
\textit{road-u

In [29]:
len(scores)

15

In [9]:
qa.db['airplane-in_front_of-building']

['a_airport_IMG_2368.jpg',
 'a_airport_airport_denver_photo20b.jpg',
 'a_arrival_gate_outdoor_arrival_gate_000052.jpg',
 'a_arrival_gate_outdoor_arrivalgateoutdoor04.jpg',
 's_skyscraper_high-rise_000008.jpg',
 's_skyscraper_tallbuilding_sky34.jpg']

In [10]:
print "\n".join(kb.ontology_by_image('a_arrival_gate_outdoor_arrival_gate_000052.jpg'))

NameError: name 'kb' is not defined

In [24]:
kb.tptp_by_image('a_airport_IMG_2368.jpg')

['fof(kb_IRRC_84,axiom,(( s__instance(s__building0__m,s__Building) ))).',
 'fof(kb_IRRC_85,axiom,(( s__instance(s__Building, s__SetOrClass) ))).',
 'fof(kb_IRRC_86,axiom,(( s__instance(s__sky0__m,s__Sky) ))).',
 'fof(kb_IRRC_87,axiom,(( s__instance(s__Sky, s__SetOrClass) ))).',
 'fof(kb_IRRC_88,axiom,(( s__instance(s____background__0__m,s____Background__) ))).',
 'fof(kb_IRRC_89,axiom,(( s__instance(s____Background__, s__SetOrClass) ))).',
 'fof(kb_IRRC_90,axiom,(( s__instance(s__airplane0__m,s__Airplane) ))).',
 'fof(kb_IRRC_91,axiom,(( s__instance(s__Airplane, s__SetOrClass) ))).',
 'fof(kb_IRRC_92,axiom,(( s__instance(s__road0__m,s__Road) ))).',
 'fof(kb_IRRC_93,axiom,(( s__instance(s__Road, s__SetOrClass) ))).',
 'fof(kb_IRRC_94,axiom,(( s__instance(s__ground0__m,s__Ground) ))).',
 'fof(kb_IRRC_95,axiom,(( s__instance(s__Ground, s__SetOrClass) ))).',
 'fof(kb_IRRC_96,axiom,(( s__orientation(s____background__0__m,s__airplane0__m,s__Under) ))).',
 'fof(kb_IRRC_97,axiom,(( s__orientat

In [26]:
kb.tptp_query('building', 'airplane', 'behind')

'fof(conj1,conjecture, ( (? [V__X,V__Y] : (s__instance(V__X,s__Building) & s__instance(V__Y,s__Airplane) & s__orientation(V__X,V__Y,s__Behind))) )).\n               '

In [17]:
retrieved['building-behind-airplane']

[u'a_arrival_gate_outdoor_arrival_gate_000052.jpg',
 u'a_airport_IMG_2080.jpg',
 u'o_opera_outdoor_opera_house_000720.jpg',
 u'a_airport_IMG_0299.jpg',
 u'a_airport_IMG_1550.jpg',
 u'a_airport_IMG_0020.jpg']

In [17]:
kb = KnowledgeBase(pd.read_csv('../data/preposition/index-0.900.csv'), 
                   ltb_runner='../data/prover/E/PROVER/e_ltb_runner', 
                   eprover='../data/prover/E/PROVER/eprover', 
                   sumo='/Users/danilonunes/Documents/sigma/run/KBs/SUMO.tptp',
                   user_assertions='/Users/danilonunes/Documents/sigma/onto/SUMO_UserAssertions.tptp')

In [18]:
# img = retrieved['building-behind-airplane'][0]
# print img
kb.prover('a_airport_IMG_2368.jpg', 'building-behind-airplane', verbose=True)

fof(kb_IRRC_0,axiom,(( s__instance(s__building1083__m,s__Building) ))).
fof(kb_IRRC_1,axiom,(( s__instance(s__Building, s__SetOrClass) ))).
fof(kb_IRRC_2,axiom,(( s__instance(s__sky1083__m,s__Sky) ))).
fof(kb_IRRC_3,axiom,(( s__instance(s__Sky, s__SetOrClass) ))).
fof(kb_IRRC_4,axiom,(( s__instance(s__airplane1083__m,s__Airplane) ))).
fof(kb_IRRC_5,axiom,(( s__instance(s__Airplane, s__SetOrClass) ))).
fof(kb_IRRC_6,axiom,(( s__instance(s__road1083__m,s__Road) ))).
fof(kb_IRRC_7,axiom,(( s__instance(s__Road, s__SetOrClass) ))).
fof(kb_IRRC_8,axiom,(( s__instance(s__ground1083__m,s__Ground) ))).
fof(kb_IRRC_9,axiom,(( s__instance(s__Ground, s__SetOrClass) ))).
fof(kb_IRRC_10,axiom,(( s__orientation(s__airplane1083__m,s__building1083__m,s__In) ))).
fof(kb_IRRC_11,axiom,(( s__orientation(s__airplane1083__m,s__ground1083__m,s__Behind) ))).
fof(kb_IRRC_12,axiom,(( s__orientation(s__airplane1083__m,s__road1083__m,s__On) ))).
fof(kb_IRRC_13,axiom,(( s__orientation(s__airplane1083__m,s__sky1083

True

In [ ]:
proof = kb.runquery('building-behind-airplane', verbose=False)


Something went wrong with image: a_access_road_slip_road_000012.jpg, index: 1024
Something went wrong with image: s_student_residence_student_residence_000012.jpg, index: 1355


In [34]:
dfx = pd.read_csv('../data/preposition/index-0.900.csv')

In [39]:
dfx[dfx.object1 != '__background__'][dfx.object2 != '__background__'].head()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,Unnamed: 0,preposition,object1,object2,score,image
6,6.0,on,building,grass,0.903399765491,a_abbey_abbey_000001.jpg
7,7.0,behind,building,path,0.440202414989,a_abbey_abbey_000001.jpg
8,8.0,under,building,sky,0.997193396091,a_abbey_abbey_000001.jpg
9,9.0,on,building,tree,0.463992267847,a_abbey_abbey_000001.jpg
11,11.0,below,grass,building,0.330984830856,a_abbey_abbey_000001.jpg


In [40]:
batch = "\n".join(["% SZS start BatchConfiguration",
                   "division.category LTB.SMO",
                   "output.required Assurance",
                   "output.desired Proof Answer",
                   "limit.time.problem.wc 120",
                   "% SZS end BatchConfiguration",
                   "% SZS start BatchIncludes",
                   "include('{tptp}').",
                   "include('{user}').",
                   "include('{sumo}').",
                   "% SZS end BatchIncludes",
                   "% SZS start BatchProblems",
                   "{problems} {answers}",
                   "% SZS end BatchProblems"])

In [42]:
print batch

% SZS start BatchConfiguration
division.category LTB.SMO
output.required Assurance
output.desired Proof Answer
limit.time.problem.wc 120
% SZS end BatchConfiguration
% SZS start BatchIncludes
include('{tptp}').
include('{user}').
include('{sumo}').
% SZS end BatchIncludes
% SZS start BatchProblems
{problems} {answers}
% SZS end BatchProblems
